In [1]:
# リスト 2.1.5
# Wikipediaのサマリー文の入手（日本語版に言語固定し、特定項目の要約テキストを取得する最小例）
# ------------------------------------------------------------------------------------------------
# 目的:
#   - Pythonの `wikipedia` ライブラリを用いて、日本語版Wikipediaから「草津温泉」のサマリー（要約）を取得し、標準出力へ表示する。
#
# 背景（理論的ポイント）:
#   - `wikipedia` は MediaWiki API を内部で叩き、ページのメタ情報や本文要約を高レベル関数で返すラッパである。
#   - `set_lang("ja")` により、照会先を日本語版Wikipediaに固定する（同名語でも各言語版でページの構成や曖昧さが異なるため、再現性の観点で重要）。
#   - `summary(title, auto_suggest=False)` は厳密一致を基本とし、内部のオートサジェスト（あいまい検索）を無効化する。
#       - 再現性の利点: 実行環境や時期によりサジェスト結果が変化する非決定性を抑制できる。
#       - 代償       : 見出し語が完全一致しない場合に `PageError` や `DisambiguationError` が発生しやすい。
#   - 「サマリー」は Wikipedia 側が定める “lead section” から抽出されるため、文数や長さはページ編集状況に依存して変動しうる。
#     再現性をさらに高めるには、`sentences=` 引数で取得文数を固定する、あるいは取得日時・版ID（oldid）の記録を検討する。
#
# 注意:
#   - 実運用ではネットワーク例外・曖昧さ例外（`wikipedia.exceptions.DisambiguationError`）・ページ不存在例外（`wikipedia.exceptions.PageError`）の捕捉を推奨。
#   - ライセンス順守（出典表記）は再配布や公開時に必要となる場合があるため、サマリー利用ポリシーを確認すること。

import wikipedia  # pip install wikipedia

# 参照する言語版を日本語に固定（デフォルトは英語版）
wikipedia.set_lang("ja")

# 見出し語「草津温泉」のサマリーを取得
# - auto_suggest=False によって曖昧検索を避け、厳密一致を要求する。
# - ページ名が曖昧・派生表記（例: 「草津温泉 (群馬県)」）の場合は例外が起きうる。
text = wikipedia.summary("草津温泉", auto_suggest=False)

# 取得したサマリーを出力（文字列）
print(text)

草津温泉（くさつおんせん）は、群馬県吾妻郡草津町草津界隈（江戸時代における上野国吾妻郡草津村界隈、幕藩体制下の上州御料草津村界隈〈初期は沼田藩知行〉）に所在する温泉である。草津白根山東麓に位置する。
毎分3万2300リットル以上の自然湧出量は日本一。室町時代には万里集九が有馬温泉や下呂温泉とともに「三名泉」とし、江戸時代には林羅山もこれらの三温泉を「天下の三名泉」と記した（日本三名泉） (cf. 1502,1662)。
江戸時代後期以降何度も作られた温泉番付の格付では、当時の最高位である大関（草津温泉は東大関）が定位置であった (cf. 1817)。


In [2]:
# リスト 2.1.6
# Wikipediaの全文の入手（日本語版の特定ページを取得し、本文テキストを出力する最小例）
# ------------------------------------------------------------------------------------------------
# 目的:
#   - Pythonの `wikipedia` ライブラリを用い、日本語版Wikipediaで「草津温泉」のページ本文（content）を取得して表示する。
#
# 理論的ポイント（NLP/再現性の観点）:
#   - `wikipedia.set_lang("ja")` により API の照会先を日本語版に固定する。多言語版でページ構造や曖昧さが異なるため、これは再現性の前提になる。
#   - `wikipedia.page(title, auto_suggest=False)` は「厳密一致」を志向する。サジェストを無効化することで、実行時期や環境差による非決定性を抑える一方、
#     見出し語が微妙に異なる場合（例: 括弧付き表記）に例外が発生しやすくなる。
#   - `page.content` は「表示用に整形済みのプレーンテキスト」（見出し・段落を含むが、テンプレ・表・注釈の構造は多くがフラット化）であり、
#     「ウィキテキスト（生のマークアップ）」そのものではない。構造的解析が目的なら MediaWiki API の別エンドポイントやパーサを検討する。
#   - Wikipediaのページ内容は編集で変動する（時変データ）。厳密な再現性が必要なら、版ID（oldid）で固定して取得する設計が望ましい。
#     ただし `wikipedia` ライブラリは oldid 指定に制限があるため、必要に応じて MediaWiki API を直接叩く実装に切り替える。
#   - 例外クラス（理論上の発生要因）:
#       * wikipedia.exceptions.DisambiguationError : 曖昧さ回避ページに遭遇
#       * wikipedia.exceptions.PageError           : ページ不存在
#       * wikipedia.exceptions.HTTPTimeoutError    : ネットワーク/応答遅延
#     本スニペットは最小例のため例外処理を省くが、実運用では捕捉してリトライや候補提示を行うのが堅牢。
#   - ライセンス実務: Wikipedia本文は CC BY-SA。公開・再配布・学術図表化の際は出典明示と継承を満たす必要がある。
#
# 補足（NLP利用時の注意）:
#   - 抽出文は改版で変わるため、ベンチマークや再学習では「取得日時」「版ID」「セクション境界」などのメタデータを一緒に保存し、
#     文字正規化（NFC/NFKC）、文分割器、トークナイザの設定も固定しておくと再現性が上がる。

import wikipedia  # pip install wikipedia

# 参照するウィキペディアの言語を日本語に固定（デフォルトは英語）
wikipedia.set_lang("ja")

# ページオブジェクトを取得
# - auto_suggest=False により、曖昧検索/自動補正に依存しない決定的な解決を試みる
# - 見出し語が厳密一致しない場合は例外（PageError/DisambiguationError）が起きうる
page = wikipedia.page("草津温泉", auto_suggest=False)

# ページ本文（プレーンテキスト）を出力
# - 見出しや改行を含む。表・注釈・テンプレの構造は多くが落ちる点に留意
print(page.content)

# （運用メモ）
#  - 必要に応じて以下のようなメタ情報も活用できる（例: page.title, page.url, page.references, page.links, page.categories）
#  - 例外処理の骨格（実装例・コメントのみ）:
#      try:
#          page = wikipedia.page('草津温泉', auto_suggest=False)
#      except wikipedia.exceptions.DisambiguationError as e:
#          # 候補 e.options から選択・再問い合わせ
#          pass
#      except wikipedia.exceptions.PageError:
#          # 見出し語の表記ゆれに対応（例: 括弧付き表記）や検索に切り替え
#          pass

草津温泉（くさつおんせん）は、群馬県吾妻郡草津町草津界隈（江戸時代における上野国吾妻郡草津村界隈、幕藩体制下の上州御料草津村界隈〈初期は沼田藩知行〉）に所在する温泉である。草津白根山東麓に位置する。
毎分3万2300リットル以上の自然湧出量は日本一。室町時代には万里集九が有馬温泉や下呂温泉とともに「三名泉」とし、江戸時代には林羅山もこれらの三温泉を「天下の三名泉」と記した（日本三名泉） (cf. 1502,1662)。
江戸時代後期以降何度も作られた温泉番付の格付では、当時の最高位である大関（草津温泉は東大関）が定位置であった (cf. 1817)。


== 名称 ==
「草津温泉（くさつおんせん）」も古来からの名称であるが、かつては、「草津湯/草津の湯（くさつのゆ）」、あるいは、上野国の異称である「上州」を冠して「上州草津湯/上州草津の湯」と呼ぶことが多かった。現在でもこれらを踏襲した雅称「草津の湯」「上州草津の湯」は頻用される。また、現在では「上州草津温泉」という名称も用いられるが、この表現は雅称的ニュアンスのほかに、他地域の「草津」や「草津温泉」という紛らわしい地名（※「#上州草津と他の草津」節を参照）と明確に区別する意図を含んでいる場合がある。
当地における「草津」という地名の語源は、温泉の硫化水素臭の強いがゆえに、「臭水（くさみず、くさうず、くそうず）」にあるとされる。また、臭處（くさと）という説もある。草津山光泉寺の縁起は、『大般若波羅蜜多経』（通称・大般若経）の一節「南方有名湯是草津湯」が由来であると説いているが、大般若経にはこのような節はなく、俗説である。同寺には、源頼朝が当地を訪ねた折りに、草を刈ったところ湯が出たという話も伝わっているが、後述するように史実性は疑わしく民間語源の可能性がある。
なお、草津温泉を、上毛かるたの「く」の札で「草津（くさづ）よいとこ薬の温泉（いでゆ）」と歌っているのは、地元で「草津」を「くさづ」と読むからで、温泉水の持つ強い硫化水素臭から「くそうづ」と呼ばれたことが今日の地名の由来であるという説がある。
古くから薬湯と知られており、恋の病以外は全て効くと言い伝えられている。


== 地理・地質 ==


=== 地理 ===
標高1100～1200mの高地に温泉街が広がっている。北西部には、草津白根山（白根山〈2,1